In [1]:
import pandas as pd
import wandb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: minjabenho. Use `wandb login --relogin` to force relogin


True

In [3]:
# Datasets
BASELINE_TRAIN = "data/train/baseline_train.csv"
BASELINE_VAL = "data/train/baseline_val.csv"
TRUNCATED_BASELINE_TRAIN = "data/train/baseline_truncated_train.csv"
TRUNCATED_BASLINE_VAL = "data/train/baseline_truncated_val.csv"
BASELINE_W_FEAT_ENG_TRAIN = "data/train/baseline_w_feature_eng_train.csv"
BASELINE_W_FEAT_ENG_VAL = "data/train/baseline_w_feature_eng_val.csv"
PPSM_BASELINE_TRAIN = "data/train/ppsm_baseline_train.csv"
PPSM_BASELINE_VAL = "data/train/ppsm_baseline_val.csv"
PPSM_FEAT_ENG_TRAIN = "data/train/feature_eng_ppsm_train.csv"
PPSM_FEAT_ENG_VAL = "data/train/feature_eng_ppsm_val.csv"

### Model For the Baseline Dataset

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric


In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [20]
    },
    'max_depth': {
        'values': [3, 6]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_TRAIN
VAL_DATA = BASELINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

y_train = train_set["monthly_rent"]
X_train = train_set.drop(columns=['monthly_rent'])

y_test = val_set['monthly_rent']
X_test = val_set.drop(columns=['monthly_rent'])


In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = GradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})



In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_baseline")
wandb.agent(sweep_id, train(X_train, y_train, X_test, y_test), count=20)

### Model for the Truncated Baseline Set

In [4]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [20]
    },
    'max_depth': {
        'values': [3, 6]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [6]:
TRAIN_DATA = TRUNCATED_BASELINE_TRAIN
VAL_DATA = TRUNCATED_BASLINE_VAL

In [7]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

y_train = train_set["monthly_rent"]
X_train = train_set.drop(columns=['monthly_rent'])

y_test = val_set['monthly_rent']
X_test = val_set.drop(columns=['monthly_rent'])

In [8]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = GradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [9]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_truncated_baseline")
wandb.agent(sweep_id, train, count=20)

Create sweep with ID: iwn1amxo
Sweep URL: https://wandb.ai/minjabenho/hdb_rental_prices_truncated_baseline/sweeps/iwn1amxo


wandb: Agent Starting Run: 26fia3s7 with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 6
wandb: 	min_impurity_decrease: 0.2
wandb: 	min_samples_leaf: 20
wandb: 	min_samples_split: 32
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 200
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


coefficient_of_determination,▁
mse,▁
coefficient_of_determination,0.53993
mse,0.22555


wandb: Agent Starting Run: 86befci5 with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 6
wandb: 	min_impurity_decrease: 0.2
wandb: 	min_samples_leaf: 20
wandb: 	min_samples_split: 16
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 300
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Ctrl + C detected. Stopping sweep.


### Model for the Feature Engineering Set

In [ ]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [20]
    },
    'max_depth': {
        'values': [3, 6]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = BASELINE_W_FEAT_ENG_TRAIN
VAL_DATA = BASELINE_W_FEAT_ENG_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

y_train = train_set["monthly_rent"]
X_train = train_set.drop(columns=['monthly_rent'])

y_test = val_set['monthly_rent']
X_test = val_set.drop(columns=['monthly_rent'])

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = GradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_feature_eng")
wandb.agent(sweep_id, train(X_train, y_train, X_test, y_test), count=20)

### Model for the Price Per Square Metre Set

In [22]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

wandb: ERROR Problem finishing run
Exception in thread Thread-51 (_run_job):
Traceback (most recent call last):
  File "/home/benhgm/miniconda3/envs/agents/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2177, in _atexit_cleanup
    self._on_finish()
  File "/home/benhgm/miniconda3/envs/agents/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2412, in _on_finish
    self._footer_exit_status_info(
  File "/home/benhgm/miniconda3/envs/agents/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 3352, in _footer_exit_status_info
    printer.display(f'{" ".join(info)}')
  File "/home/benhgm/miniconda3/envs/agents/lib/python3.10/site-packages/wandb/sdk/lib/printer.py", line 68, in display
    self._display(text, level=level, default_text=default_text)
  File "/home/benhgm/miniconda3/envs/agents/lib/python3.10/site-packages/wandb/sdk/lib/printer.py", line 247, in _display
    self._display_fn_mapping(level)(text)
  File "/home/benhgm/miniconda3/envs/agents/lib/python

In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [20]
    },
    'max_depth': {
        'values': [3, 6]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
TRAIN_DATA = PPSM_BASELINE_TRAIN
VAL_DATA = PPSM_BASELINE_VAL

In [ ]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

y_train = train_set["rent_per_sqm"]
X_train = train_set.drop(columns=['rent_per_sqm'])

y_test = val_set['rent_per_sqm']
X_test = val_set.drop(columns=['rent_per_sqm'])

In [ ]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = GradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_ppsm_baseline")
wandb.agent(sweep_id, train, count=20)

### Model for the Price Per Square Metre Set with Feature Engineering

In [4]:
sweep_config = {
    'method': 'bayes'
}
metric = {
    'name': 'mse',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'learning_rate': {
        'values': [0.01, 0.1]
    },
    'n_estimators': {
        'values': [100, 200, 300, 500]
    },
    'min_samples_split': {
        'values': [4, 16, 32]
    },
    'min_samples_leaf': {
        'values': [10, 20, 30]
    },
    'max_depth': {
        'values': [3, 6, 9]
    },
    'min_impurity_decrease': {
        'values': [0.1, 0.2]
    },
    'min_weight_fraction_leaf': {
        'values': [0.0, 0.1]
    },
    'subsample': {
        'values': [0.75, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [6]:
TRAIN_DATA = PPSM_FEAT_ENG_TRAIN
VAL_DATA = PPSM_FEAT_ENG_VAL

In [7]:
train_set = pd.read_csv(TRAIN_DATA)
val_set = pd.read_csv(VAL_DATA)

y_train = train_set["rent_per_sqm"]
X_train = train_set.drop(columns=['rent_per_sqm'])

y_test = val_set['rent_per_sqm']
X_test = val_set.drop(columns=['rent_per_sqm'])

In [8]:
def train():
    with wandb.init():
        config = wandb.config

        regressor = GradientBoostingRegressor(
            random_state=42,
            learning_rate=config.learning_rate,
            n_estimators=config.n_estimators,
            max_depth=config.max_depth,
            subsample=config.subsample,
            min_samples_leaf=config.min_samples_leaf,
            min_samples_split=config.min_samples_split,
            min_impurity_decrease=config.min_impurity_decrease,
            min_weight_fraction_leaf=config.min_weight_fraction_leaf)
        regressor.fit(X_train, y_train)
        score = regressor.score(X_test, y_test)
        y_pred = regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

    
        # feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
        # plt.barh(X_train.columns, feat_importances.nlargest(10))
        # wandb.log({'feature_importances': plt})

        # wandb.sklearn.plot_regressor(regressor, X_train, X_test, y_train, y_test, 'GradientBoost')
        wandb.log(data={"mse": mse, "coefficient_of_determination": score})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="hdb_rental_prices_ppsm_feat_eng")
wandb.agent(sweep_id, train, count=20)

Create sweep with ID: 221messt
Sweep URL: https://wandb.ai/minjabenho/hdb_rental_prices_ppsm_feat_eng/sweeps/221messt


wandb: Agent Starting Run: 2s5o6nsq with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 3
wandb: 	min_impurity_decrease: 0.1
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 4
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 300
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


coefficient_of_determination,▁
mse,▁
coefficient_of_determination,0.60061
mse,31.94579


wandb: Agent Starting Run: kcuu803a with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 3
wandb: 	min_impurity_decrease: 0.1
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 16
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 300
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


coefficient_of_determination,▁
mse,▁
coefficient_of_determination,0.60061
mse,31.94579


...

wandb: Agent Starting Run: hw4tukyu with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 3
wandb: 	min_impurity_decrease: 0.2
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 4
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 300
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


coefficient_of_determination,▁
mse,▁
coefficient_of_determination,0.60061
mse,31.94579


wandb: Agent Starting Run: qe7jqgg3 with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 9
wandb: 	min_impurity_decrease: 0.2
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 32
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 100
wandb: 	subsample: 0.75
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


coefficient_of_determination,▁
mse,▁
coefficient_of_determination,0.60643
mse,31.4798


wandb: Agent Starting Run: ny21ku9l with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 9
wandb: 	min_impurity_decrease: 0.2
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 16
wandb: 	min_weight_fraction_leaf: 0
wandb: 	n_estimators: 300
wandb: 	subsample: 0.75
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Ctrl + C detected. Stopping sweep.
